# Union with DataSets

## Load the necessary datasets

### Setup and Configuration

In [1]:
# Import necessary libraries
import os
# import numpy as np
# import pandas as pd
import geopandas as gpd

from shapely.validation import make_valid
from shapely import set_precision
# from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon

# from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings('ignore')

import _params as params

# Configuration
TARGET_CRS = 26912
GRID = 0.1 # meters in EPSG:26912; 0.1–1.0 works well

### Prepare Processing Functions

In [2]:
def load_and_validate(file_path):
    """Load a file, ensure CRS, fix invalid/empty geometries."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing expected layer: {file_path}")
    gdf = gpd.read_file(file_path)
    if gdf.empty:
        raise ValueError(f"Layer has no features: {file_path}")

    # normalize CRS to WGS84 then project to target for overlay robustness
    if gdf.crs is None:
        # assume WGS84 if unspecified (adjust if your sources differ)
        gdf.set_crs(epsg=4326, inplace=True)
    gdf = gdf.to_crs(epsg=TARGET_CRS)
    gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()].copy()

    # Fix any invalid geometries
    invalid_count = (~gdf.is_valid).sum()
    if invalid_count > 0:
        print(f"   🔧 Fixing {invalid_count} invalid geometries")
        gdf["geometry"] = gdf["geometry"].apply(make_valid)
        gdf["geometry"] = gdf["geometry"].apply(lambda g: set_precision(g, GRID))
    return gdf

In [3]:
def ensure_multipolygon(gdf):
    """Convert all geometries to MultiPolygon, filtering out non-polygon types."""
    def _to_multipolygon(geom):
        if geom is None:
            return None
        if geom.geom_type == "Polygon":
            return MultiPolygon([geom])
        elif geom.geom_type == "MultiPolygon":
            return geom
        else:
            # Drop non-polygon geometries (points, lines, etc.)
            return None

    gdf = gdf.copy()
    gdf["geometry"] = gdf["geometry"].apply(_to_multipolygon)
    gdf = gdf[gdf.geometry.notna() & ~gdf.geometry.is_empty].copy()

    # Verify all geometries are MultiPolygon
    if not gdf.empty and not all(gdf.geometry.geom_type == 'MultiPolygon'):
        print(f"Warning: Not all geometries converted to MultiPolygon")
        gdf = gdf[gdf.geometry.geom_type == 'MultiPolygon'].copy()

    return gdf

In [4]:
def add_buffer_suffix(file_path):
    """Add '_Buffers' to filename before extension."""
    base, ext = os.path.splitext(file_path)
    return f"{base}_Buffers{ext}"

### Load All Layers

In [5]:
# Files that will be processed as-is (no buffering needed)
regular_files = [
    params.strCommunitiesOut,
    params.strCentersOut,
    params.strTAZwithATOOut,
    params.strQOZOut,
    params.strParkAccessibilityOut
]

# Files that need buffering (points/lines → polygons)
files_to_buffer = [
    params.strInterchangesOut_Cur,
    params.strInterchangesOut_Fut,
    params.strTransitOut_Lcl,
    params.strTransitOut_BrtCur,
    params.strTransitOut_BrtFut,
    params.strTransitOut_LrtCur,
    params.strTransitOut_LrtFut,
    params.strTransitOut_CrtCur,
    params.strTransitOut_CrtFut,
    params.strChildCareOut,
    params.strHealthCareOut,
    params.strSchoolsOut_RegPub,
    params.strSchoolsOut_HighEd,
    params.strGroceryOut,
    params.strCommunityCenterOut,
    params.strPathsOut_Cur,
    params.strPathsOut_Fut,
    params.strATCycleTracksOut_Cur,
    params.strATCycleTracksOut_Fut
]

In [6]:
# Initiatize all layers
all_layers = {}

In [7]:
# Load regular files
print("Loading regular files...")
for file_path in regular_files:
    try:
        name = os.path.splitext(os.path.basename(file_path))[0]
        gdf = load_and_validate(file_path)
        gdf = ensure_multipolygon(gdf)  # Convert to MultiPolygon in one step
        all_layers[name] = gdf
        print(f"✅ Loaded {name} ({len(gdf)} MultiPolygons)")
    except Exception as e:
        print(f"⚠ Error processing {file_path}: {e}")
        continue

Loading regular files...
✅ Loaded Communities (75 MultiPolygons)
✅ Loaded Centers (351 MultiPolygons)
✅ Loaded TAZWithATOScores (2858 MultiPolygons)
✅ Loaded UtahQualifiedOpportunityZones (28 MultiPolygons)
   🔧 Fixing 1 invalid geometries
✅ Loaded ParksAndOpenSpace (1 MultiPolygons)


In [8]:
# Load buffered files
print("\nLoading buffered files...")
for file_path in files_to_buffer:
    try:
        buffered_path = add_buffer_suffix(file_path)
        name = os.path.splitext(os.path.basename(buffered_path))[0]
        gdf = load_and_validate(buffered_path)
        gdf = ensure_multipolygon(gdf)  # Convert to MultiPolygon in one step
        all_layers[name] = gdf
        print(f"✅ Loaded {name} ({len(gdf)} MultiPolygons)")
    except Exception as e:
        print(f"⚠ Error processing {file_path}: {e}")
        continue


Loading buffered files...
✅ Loaded Interchanges_Buffers (984 MultiPolygons)
✅ Loaded Interchanges_Future_Buffers (96 MultiPolygons)
✅ Loaded LocalBusStops_Buffers (5299 MultiPolygons)
✅ Loaded BRTStops_Buffers (40 MultiPolygons)
✅ Loaded BRTStops_Future_Buffers (118 MultiPolygons)
✅ Loaded LRTStops_Buffers (56 MultiPolygons)
✅ Loaded LRTStops_Future_Buffers (73 MultiPolygons)
✅ Loaded CRTStops_Buffers (15 MultiPolygons)
✅ Loaded CRTStops_Future_Buffers (9 MultiPolygons)
✅ Loaded ChildCare_Buffers (740 MultiPolygons)
✅ Loaded HealthCare_Buffers (472 MultiPolygons)
✅ Loaded SchoolsRegPublic_Buffers (2060 MultiPolygons)
✅ Loaded SchoolsHigherEd_Buffers (130 MultiPolygons)
✅ Loaded GroceryStores_Buffers (874 MultiPolygons)
✅ Loaded CommunityCenter_Buffers (350 MultiPolygons)
✅ Loaded ATPaths_Buffers (16064 MultiPolygons)
✅ Loaded ATPaths_Future_Buffers (1378 MultiPolygons)
✅ Loaded ATCycleTracks_Buffers (90 MultiPolygons)
✅ Loaded ATCycleTracks_Future_Buffers (1574 MultiPolygons)


In [9]:
# Load parcel layer separately
print("\nLoading parcel layer...")
try:
    parcels = load_and_validate(params.strParcelsOut)
    parcels = ensure_multipolygon(parcels)  # Convert to MultiPolygon in one step
    print(f"✅ Loaded Parcels ({len(parcels)} MultiPolygons)")
except Exception as e:
    print(f"⚠ Error loading parcels: {e}")
    parcels = None

print(f"\n🎯 Total layers ready for union: {len(all_layers)} + Parcels")


Loading parcel layer...
   🔧 Fixing 358 invalid geometries
✅ Loaded Parcels (712179 MultiPolygons)

🎯 Total layers ready for union: 24 + Parcels


## Union Layers

### Define Union Functions

In [10]:
def clean_geometry_for_overlay(gdf, buffer_dist=0.0001):
    """Clean and prepare geometry for overlay operations."""
    # Remove empty/null geometries
    gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()].copy()

    # Fix invalid geometries
    invalid_mask = ~gdf.geometry.is_valid
    if invalid_mask.any():
        print(f"   Fixing {invalid_mask.sum()} invalid geometries")
        gdf.loc[invalid_mask, 'geometry'] = gdf.loc[invalid_mask, 'geometry'].apply(make_valid)

    # Apply tiny buffer to fix topology issues
    gdf['geometry'] = gdf.geometry.buffer(buffer_dist)

    # Force back to MultiPolygon after buffer (buffer can change geometry types)
    gdf = ensure_multipolygon(gdf)

    return gdf

In [11]:
def union_all_layers_simple(layers_dict, parcels_gdf):
    """
    Simple union approach: Use intersections instead of complex overlay operations.
    """
    print("🔄 Starting simple union operation...")

    if parcels_gdf is None or parcels_gdf.empty:
        raise ValueError("Parcels layer is required and cannot be empty")

    # Start with clean parcels
    print("📦 Preparing parcels as base layer...")
    result_gdf = clean_geometry_for_overlay(parcels_gdf.copy())
    print(f"   Parcels prepared: {len(result_gdf)} features")

    # Add indicator columns for all layers
    for layer_name in layers_dict.keys():
        result_gdf[f'has_{layer_name}'] = 0

    # Process each layer by finding intersections
    layer_count = 0
    total_layers = len(layers_dict)

    for layer_name, layer_gdf in layers_dict.items():
        layer_count += 1
        print(f"🔄 Processing layer {layer_count}/{total_layers}: {layer_name} ({len(layer_gdf)} features)")

        if layer_gdf.empty:
            print(f"   Skipping empty layer: {layer_name}")
            continue

        try:
            # Clean the overlay layer
            overlay_gdf = clean_geometry_for_overlay(layer_gdf.copy())
            print(f"   Cleaned overlay layer: {len(overlay_gdf)} features")

            # Ensure same CRS
            if overlay_gdf.crs != result_gdf.crs:
                overlay_gdf = overlay_gdf.to_crs(result_gdf.crs)

            # Create spatial index for efficiency
            spatial_index = overlay_gdf.sindex

            # Find intersecting parcels
            intersecting_indices = []
            for idx, parcel_geom in result_gdf.geometry.items():
                possible_matches_index = list(spatial_index.intersection(parcel_geom.bounds))
                if possible_matches_index:
                    for match_idx in possible_matches_index:
                        if parcel_geom.intersects(overlay_gdf.iloc[match_idx].geometry):
                            intersecting_indices.append(idx)
                            break

            # Mark intersecting parcels
            result_gdf.loc[intersecting_indices, f'has_{layer_name}'] = 1

            print(f"   Found {len(intersecting_indices)} intersecting parcels")

        except Exception as e:
            print(f"⚠ Error processing {layer_name}: {e}")
            print("   Continuing with remaining layers...")
            continue

    print(f"✅ Simple union complete! Final result: {len(result_gdf)} features")
    return result_gdf

### Execute Union Operations

In [ ]:
if parcels is not None and not parcels.empty:
    try:
        # Perform the union operation
        gdf_unioned = union_all_layers_simple(all_layers, parcels)

        # Add summary statistics
        layer_columns = [col for col in gdf_unioned.columns if col.startswith('has_')]
        gdf_unioned['total_layers'] = gdf_unioned[layer_columns].sum(axis=1)

        print(f"\n📊 Union Summary:")
        print(f"   Total output features: {len(gdf_unioned):,}")
        print(f"   Average layers per feature: {gdf_unioned['total_layers'].mean():.2f}")
        print(f"   Max layers in single feature: {gdf_unioned['total_layers'].max()}")

        # Show distribution of layer counts
        layer_dist = gdf_unioned['total_layers'].value_counts().sort_index()
        print(f"   Layer distribution:")
        for layers, count in layer_dist.head(10).items():
            print(f"     {int(layers)} layers: {count:,} parcels")

    except Exception as e:
        print(f"❌ Union operation failed: {e}")
        gdf_unioned = None
else:
    print("❌ Cannot perform union: Parcels layer is missing or empty") # 1736687 esimated total # Run Time ~ 182m 33.9s
    gdf_unioned = None

🔄 Starting simple union operation...
📦 Preparing parcels as base layer...
   Parcels prepared: 710220 features
🔄 Processing layer 1/24: Communities (75 features)
   Cleaned overlay layer: 75 features
   Found 671830 intersecting parcels
🔄 Processing layer 2/24: Centers (351 features)
   Cleaned overlay layer: 351 features
   Found 46356 intersecting parcels
🔄 Processing layer 3/24: TAZWithATOScores (2858 features)
   Cleaned overlay layer: 2858 features
   Found 706586 intersecting parcels
🔄 Processing layer 4/24: UtahQualifiedOpportunityZones (28 features)
   Cleaned overlay layer: 28 features
   Found 34067 intersecting parcels
🔄 Processing layer 5/24: ParksAndOpenSpace (1 features)
   Cleaned overlay layer: 1 features
   Found 327415 intersecting parcels
🔄 Processing layer 6/24: Interchanges_Buffers (984 features)
   Cleaned overlay layer: 984 features
   Found 400979 intersecting parcels
🔄 Processing layer 7/24: Interchanges_Future_Buffers (96 features)
   Cleaned overlay layer: 96

### Export Results

In [13]:
if gdf_unioned is not None and not gdf_unioned.empty:
    print("\n💾 Exporting results...")

    # Reproject to WGS84 for export
    gdf_unioned = gdf_unioned.to_crs(epsg=4326)

    # Export to GeoJSON
    output_path = os.path.join(params.dirIntermediate, "UNIONED.geojson")

    try:
        gdf_unioned.to_file(output_path, driver="GeoJSON")
        print(f"✅ Union result exported to: {output_path}")
        print(f"   File size: {os.path.getsize(output_path) / (1024*1024):.1f} MB")

        # Also export a sample for inspection
        sample_path = os.path.join(params.dirIntermediate, "UNIONED_sample.geojson")
        gdf_unioned.head(1000).to_file(sample_path, driver="GeoJSON")
        print(f"   Sample (first 1000 features) exported to: {sample_path}")

    except Exception as e:
        print(f"❌ Export failed: {e}")

else:
    print("❌ No results to export")


💾 Exporting results...
✅ Union result exported to: d:\GitHub\MAP-Housing-ATO-Calculator\intermediate\UNIONED.geojson
   File size: 4363.0 MB
   Sample (first 1000 features) exported to: d:\GitHub\MAP-Housing-ATO-Calculator\intermediate\UNIONED_sample.geojson
